In [1]:
# Core dependencies for the project
!pip install --upgrade pip setuptools wheel  # Always good to start with an upgrade

# LangChain and related packages
!pip install "langchain>=0.1.12,<0.2.0"
!pip install "langchain-community>=0.0.29,<1.0.0"

# API and AI packages
!pip install anthropic
!pip install openai  # Optional, depending on your specific needs

# Web and utility packages
!pip install streamlit==1.32.2
!pip install pyngrok==7.1.5
!pip install pillow
!pip install python-dotenv
!pip install pyyaml

In [2]:
import locale
# Ensure UTF-8 encoding
locale.getpreferredencoding = lambda: "UTF-8"

import yaml
import os

# Load API credentials with error handling
try:
    # Ensure the file exists in the correct path
    with open('anthropic_api_credentials.yml', 'r') as file:
        api_creds = yaml.safe_load(file)

    # Validate that the key exists
    if 'anthropic_key' not in api_creds:
        raise KeyError("Anthropic API key not found in the credentials file")

    # Set the environment variable
    os.environ['ANTHROPIC_API_KEY'] = api_creds['anthropic_key']

except FileNotFoundError:
    print("Error: anthropic_api_credentials.yml file not found.")
    print("Please create the file with your Anthropic API key.")
except KeyError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [3]:
!pip install langchain-anthropic

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 0.3.40 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.40 which is incompatible.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.40 which is incompatible.


In [4]:
%%writefile app.py
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import streamlit as st
import yaml
import os

# Importing LangChain and Anthropic components
from langchain_anthropic import ChatAnthropic
from langchain.schema.messages import HumanMessage, AIMessage
import base64
from PIL import Image
import io

# Page configuration
st.set_page_config(
    page_title="Image Analysis Q&A with Claude",
    page_icon="🔍",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Add custom CSS for dark theme
st.markdown("""
    <style>
    .stApp {
        background-color: #0E1117;
        color: #FAFAFA;
    }

    .stButton>button {
        width: 100%;
        background-color: #262730;
        color: #FAFAFA;
        border: 1px solid #4F4F4F;
    }

    .stButton>button:hover {
        background-color: #363740;
        border: 1px solid #6F6F6F;
    }

    /* ... (rest of the CSS remains the same) ... */
    </style>
""", unsafe_allow_html=True)

# Load API credentials
def load_api_credentials():
    """Load API credentials from YAML file"""
    try:
        with open('anthropic_api_credentials.yml', 'r') as file:
            api_creds = yaml.safe_load(file)
        os.environ['ANTHROPIC_API_KEY'] = api_creds['anthropic_key']
        return True
    except Exception as e:
        st.error(f"Error loading API credentials: {e}")
        return False

def validate_image(image_file):
    """Validates the uploaded image format"""
    if image_file is None:
        return False

    try:
        file_type = image_file.type.lower()
        valid_types = ['image/jpeg', 'image/jpg', 'image/png']

        if file_type not in valid_types:
            st.error("⚠️ Please upload only JPG, JPEG or PNG images.")
            return False

        image = Image.open(image_file)
        if image_file.size > 20 * 1024 * 1024:
            st.error("⚠️ Image size too large. Please upload an image smaller than 20MB.")
            return False
        return True
    except Exception as e:
        st.error(f"⚠️ Error validating image: {str(e)}")
        return False

def encode_image(image_file):
    """Converts uploaded image to base64"""
    if image_file is not None:
        bytes_data = image_file.getvalue()
        encoded = base64.b64encode(bytes_data).decode()
        return encoded
    return None

def initialize_claude_chat(model):
    """Initializes the LangChain Claude chat"""
    try:
        return ChatAnthropic(
            model=model,
            max_tokens=1000,
            temperature=0.7
        )
    except Exception as e:
        st.error(f"⚠️ Error initializing Claude chat: {str(e)}")
        return None

def display_chat_history():
    """Displays the chat history with custom styling"""
    if "messages" in st.session_state:
        for message in st.session_state.messages:
            if isinstance(message, HumanMessage):
                # Handle multi-part messages
                content = message.content[0]['text'] if isinstance(message.content, list) else message.content
                st.markdown(f"""
                    <div class="chat-message user-message">
                        <b>You:</b> {content}
                    </div>
                """, unsafe_allow_html=True)
            elif isinstance(message, AIMessage):
                st.markdown(f"""
                    <div class="chat-message ai-message">
                        <b>Claude:</b> {message.content}
                    </div>
                """, unsafe_allow_html=True)

def main():
    # Load API credentials
    if not load_api_credentials():
        return

    # Initialize session state for memory
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    if 'current_image' not in st.session_state:
        st.session_state.current_image = None

    # Create two columns for layout
    col1, col2 = st.columns([2, 1])

    with col1:
        st.title("🔍 Image Analysis Q&A with Claude")
        st.markdown("---")

        with st.expander("ℹ️ How to use this app"):
            st.markdown("""
                1. Upload an image (JPG, JPEG, or PNG)
                2. Type your question about the image
                3. Click 'Analyze' to get insights from Claude

                **Tips:**
                - Keep images under 20MB
                - Ask clear, specific questions
                - Try questions about objects, colors, text, or activities in the image
                - The chat remembers your conversation history for context
            """)

    # Model selection
    model = st.selectbox(
        "Select Claude Model",
        [
            "claude-3-opus-20240229",
            "claude-3-sonnet-20240229",
            "claude-3-haiku-20240307"
        ],
        help="Choose the Claude model for image analysis"
    )

    # Initialize LangChain Claude chat
    chat = initialize_claude_chat(model)
    if not chat:
        return

    # Main interface
    image_file = st.file_uploader("📤 Upload an image to analyze", type=['png', 'jpg', 'jpeg'])

    if image_file and validate_image(image_file):
        # Clear message history if new image is uploaded
        if st.session_state.current_image != image_file.name:
            st.session_state.messages = []
            st.session_state.current_image = image_file.name

        st.markdown("### 📷 Uploaded Image")
        image = Image.open(image_file)
        st.image(image, caption='', use_column_width=True)

        # Display chat history
        display_chat_history()

        # Create a container for dynamic updates
        response_container = st.container()

        st.markdown("### ❓ Ask your question")
        question = st.text_input("", placeholder="What would you like to know about this image?")

        if st.button("🔍 Analyze Image", key="analyze"):
            if not question:
                st.warning("⚠️ Please enter a question about the image.")
                return

            try:
                with st.spinner("🔄 Analyzing image..."):
                    # Create message with image context
                    message = HumanMessage(
                        content=[
                            {
                                "type": "text",
                                "text": question
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/{image_file.type.split('/')[-1]};base64,{encode_image(image_file)}",
                                    "detail": "auto"
                                }
                            }
                        ]
                    )

                    # Add message to memory
                    st.session_state.messages.append(message)

                    # Get response using chat history for context
                    response = chat.invoke(st.session_state.messages)

                    # Add response to memory
                    st.session_state.messages.append(response)

                    # Update the response container
                    with response_container:
                        display_chat_history()

            except Exception as e:
                st.error(f"⚠️ An error occurred during analysis: {str(e)}")
                st.markdown("""
                    **Possible solutions:**
                    1. Check your internet connection
                    2. Verify your Anthropic API key
                    3. Try uploading a different image
                    4. If the problem persists, try again later
                """)

if __name__ == "__main__":
    main()

Writing app.py


In [5]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [6]:
from pyngrok import ngrok
import yaml
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://8d01-34-48-198-177.ngrok-free.app
